In [9]:
import os
from glob import glob
from pathlib import Path
import lib.engine as engine
from IPython.display import display
from lib.utils import ButtonCenter, HBoxAlign 
from ipywidgets import Layout, Output, Select,VBox, HBox, HTML

In [10]:
users=['rama',]
output_ds=['dogsLabeled']
DATASETS_PATH='./data/*'

In [14]:
output_init = Output(layout={'border': '0px solid transparent'})
def init_tool(DATASET_PATH, OUTPUT_PATH):
    with output_init:
        display(HBox(children=[HTML("<h1>Loading... <i class='fas fa-compact-disc fa-spin'></i></h1>",)],
        layout=Layout(height='auto', width='auto', justify_content='center')))
    
    sub_classes = ['big','medium','small']

    labels=[ 
        'Labrador Retriever',
        'French Bulldog',
        'German Shepherd',
        'Golden Retriever',
        'English Bulldog',
        'Poodle',
        'Beagle',
        'Rottweiler',
    ] 

    # Folder generation
    for l in labels:
        for c in sub_classes: 
            Path(f'{OUTPUT_PATH}/{l}/{c}').mkdir(exist_ok=True, parents=True)

    Path(f'{OUTPUT_PATH}/Discard').mkdir(exist_ok=True, parents=True)
    Path(f'{OUTPUT_PATH}/Skip').mkdir(exist_ok=True, parents=True)

    # Listing files
    toLabel = [(r, d, files) for r, d, files in os.walk(DATASET_PATH)][0][-1]
    labeled = list(filter(lambda x: len(x[1])==0, [(r, d, files) for r, d, files in os.walk(OUTPUT_PATH)]))
    to_label_imgs=[]
    for img in toLabel:
        toLabel=True
        for class_folders in labeled:
            if img in class_folders[-1]:
                toLabel=False
                break
        if toLabel:
            to_label_imgs.append(os.path.join(DATASET_PATH, img))
         
    app = engine.labeling_display(to_label_imgs, labels, sub_classes,  OUTPUT_PATH)

    with output_init:
        display(app.controls, clear=True)
    
    display(app.output_images)
    display(app.output_progress)




In [15]:
def parse_options():    
    DATASET_PATH=f'{sub_ds_options.value}'
    OUTPUT_PATH=f'./labeled_data_{output_ds_options.value}_{user_options.value}'
    init_tool(DATASET_PATH, OUTPUT_PATH)

btn_go = ButtonCenter('Continue', style='success', icon='fa-check', onClick=lambda _: parse_options())

user_options=Select(
    options=users,
    value=users[0],
    description='User:',
    disabled=False,
)
ds_options_list = glob(DATASETS_PATH)
sub_ds_options = Select(
    options=ds_options_list,
    value=ds_options_list[0],
    description='Source DS:',
    disabled=False,
)

output_ds_options=Select(
    options=output_ds,
    value=output_ds[0],
    description='Output Dataset:',
    disabled=False,
)

# initbox = HBox(children=[user_options,sub_ds_options,output_ds_options], layout=Layout(height='auto', width='auto', justify_content='center')) 
initbox = HBoxAlign([user_options,sub_ds_options,output_ds_options], justify='center')
init_grid = VBox(children=[
    initbox,
    HBoxAlign([btn_go], justify='center')
])

display(output_init)
with output_init:
    display(init_grid)

Output(layout=Layout(border='0px solid transparent'))